In [34]:
import pandas as pd
from datetime import timedelta

In [35]:
YEARS = [2016,2017,2018,2019,2020,2021,2022]

def create_peak_hours():
    all_peak_hours = []
    for year in YEARS:
        file_path = f'./data/peak-hours/{year}-peak-hours.xlsx'
        try:
            df = pd.read_excel(file_path, sheet_name=f'{year} Peak Hour Report')
            df['YEAR'] = year  # Use the trusted file-based year
            all_peak_hours.append(df)
        except FileNotFoundError:
            print(f"File not found for year {year}, skipping...")

    peak_hours_df = pd.concat(all_peak_hours, ignore_index=True)

    # Only map day and month (keep trusted YEAR column)
    day_mapping = {
        'MON': 0, 'TUE': 1, 'WED': 2, 
        'THU': 3, 'FRI': 4, 'SAT': 5, 'SUN': 6
    }

    month_mapping = {
        'JAN': 1, 'FEB': 2, 'MAR': 3, 
        'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7,
        'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
    }

    peak_hours_df['AM_DAY'] = peak_hours_df['AM_DAY'].replace(day_mapping)   
    peak_hours_df['AM_MONTH'] = peak_hours_df['AM_MONTH'].replace(month_mapping)

    return peak_hours_df


In [36]:
def add_am_datetime(peak_hours_df):
    # Filter out rows with invalid months just in case
    peak_hours_df = peak_hours_df[peak_hours_df['AM_MONTH'].between(1, 12)]

    # Sort so dates are in logical order
    peak_hours_df = peak_hours_df.sort_values(['YEAR', 'AM_MONTH', 'AM_HOUR']).reset_index(drop=True)

    # Create a dummy start date: use the first valid year and month
    start_date = pd.Timestamp(f"{peak_hours_df.loc[0, 'YEAR']}-{peak_hours_df.loc[0, 'AM_MONTH']:02d}-01")

    # Create artificial daily dates for the length of the dataframe
    peak_hours_df['AM_DATE'] = pd.date_range(start=start_date, periods=len(peak_hours_df), freq='D')

    # Add hour as a timedelta
    peak_hours_df['AM_DATETIME'] = peak_hours_df['AM_DATE'] + pd.to_timedelta(peak_hours_df['AM_HOUR'], unit='h')

    return peak_hours_df


In [40]:
df = create_peak_hours()
df = add_am_datetime(df)

print(df[['YEAR', 'AM_MONTH', 'AM_HOUR', 'AM_DATETIME']])


File not found for year 2017, skipping...
File not found for year 2018, skipping...
      YEAR  AM_MONTH  AM_HOUR         AM_DATETIME
0     2016         1        5 2016-01-01 05:00:00
1     2016         1        5 2016-01-02 05:00:00
2     2016         1        5 2016-01-03 05:00:00
3     2016         1        6 2016-01-04 06:00:00
4     2016         1        6 2016-01-05 06:00:00
...    ...       ...      ...                 ...
6794  2022        12       12 2034-08-08 12:00:00
6795  2022        12       12 2034-08-09 12:00:00
6796  2022        12       12 2034-08-10 12:00:00
6797  2022        12       12 2034-08-11 12:00:00
6798  2022        12       12 2034-08-12 12:00:00

[6799 rows x 4 columns]


/var/folders/y0/tql4kxn91m51l1_d96hty_z40000gn/T/ipykernel_8793/2408790939.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  peak_hours_df['AM_DAY'] = peak_hours_df['AM_DAY'].replace(day_mapping)
/var/folders/y0/tql4kxn91m51l1_d96hty_z40000gn/T/ipykernel_8793/2408790939.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  peak_hours_df['AM_MONTH'] = peak_hours_df['AM_MONTH'].replace(month_mapping)


In [38]:
peak_hours_df = peak_hours_df.sort_values(['YEAR', 'AM_MONTH', 'AM_HOUR']).reset_index(drop=True)

# Step 3: Build date range from a known start
# Let's assume each row is one day, starting from the earliest date
start_date = pd.Timestamp(f"{peak_hours_df.loc[0, 'YEAR']}-{peak_hours_df.loc[0, 'AM_MONTH']:02d}-01")

# Step 4: Build a real date column (optional – assumes each row is a day apart)
peak_hours_df['AM_DATE'] = pd.date_range(start=start_date, periods=len(peak_hours_df), freq='D')

# Step 5: Add the AM_HOUR as time
peak_hours_df['AM_DATETIME'] = peak_hours_df['AM_DATE'] + pd.to_timedelta(peak_hours_df['AM_HOUR'], unit='h')

In [39]:
peak_hours_df['AM_DATETIME']

0      2014-01-01 07:00:00
1      2014-01-02 07:00:00
2      2014-01-03 07:00:00
3      2014-01-04 07:00:00
4      2014-01-05 08:00:00
               ...        
6794   2032-08-08 12:00:00
6795   2032-08-09 12:00:00
6796   2032-08-10 12:00:00
6797   2032-08-11 12:00:00
6798   2032-08-12 12:00:00
Name: AM_DATETIME, Length: 6799, dtype: datetime64[ns]